In [1]:
import sys
sys.path.append("..")
import pickle

In [2]:
from games.abstract_poker import AbstractPokerRootChanceGameState
from games.algorithms import ChanceSamplingCFR, VanillaCFR
from common.constants import card_dealing, results_map
import numpy as np
import pandas as pd

from games.competition_abstract_poker import CompetitionAbstractPoker
from copy import deepcopy

In [3]:
# Hyperparameters
_NUM_TRAIN_ITERS = 100_000
_COMPETITION_ROUNDS = 100_000
_RUN_SINGLE_EXAMPLE = False

# Try single example

In [4]:
if _RUN_SINGLE_EXAMPLE:
    bots = []

    root = AbstractPokerRootChanceGameState(max_hand_strength = 3, 
                                        max_turns = 1,
                                        report = False)
    chance_sampling_cfr = ChanceSamplingCFR(root)
    chance_sampling_cfr.run(iterations = 100)
    chance_sampling_cfr.compute_nash_equilibrium()

    bots.append(chance_sampling_cfr)

    root = AbstractPokerRootChanceGameState(max_hand_strength = 3, 
                                        max_turns = 2,
                                        report = False)
    chance_sampling_cfr = ChanceSamplingCFR(root)
    chance_sampling_cfr.run(iterations = 100)
    chance_sampling_cfr.compute_nash_equilibrium()

    bots.append(chance_sampling_cfr)

    results = []
    rounds_per_competition = 1

    c = CompetitionAbstractPoker(deepcopy(bots[0]), deepcopy(bots[1]), max_turns = 2, report = True)
    result = c.compete(rounds = rounds_per_competition)
    print(f"Result is {result}")

# Granularity competition

In [5]:
def train_bots_fixed_granularity(depth,  granularity, report = True):
    bots = []

    for i in depth:
        root = AbstractPokerRootChanceGameState(max_hand_strength = granularity, 
                                            max_turns = i,
                                            report = False)
        chance_sampling_cfr = ChanceSamplingCFR(root)
        chance_sampling_cfr.run(iterations = _NUM_TRAIN_ITERS)
        chance_sampling_cfr.compute_nash_equilibrium()

        # vanilla cfr
        # vanilla_cfr_g3 = VanillaCFR(root)
        # vanilla_cfr_g3.run(iterations = _NUM_TRAIN_ITERS)
        # vanilla_cfr_g3.compute_nash_equilibrium()

        bots.append(chance_sampling_cfr)
        
    if report:
        print("------------------------------------------------------------")
        for b in bots:
            print(f"Value of game for depth {b.root.max_turns}: {b.value_of_the_game():.5f}")
        
    return bots

## Train static bots

In [6]:
low_granularity_bots = train_bots_fixed_granularity(depth = [1,2,3,4], granularity = 3)
high_granularity_bots = train_bots_fixed_granularity(depth = [1,2,3,4], granularity = 10)

------------------------------------------------------------
Value of game for depth 1: -0.02801
Value of game for depth 2: -0.02192
Value of game for depth 3: -0.02534
Value of game for depth 4: -0.02622
------------------------------------------------------------
Value of game for depth 1: -0.05296
Value of game for depth 2: -0.05229
Value of game for depth 3: -0.04946
Value of game for depth 4: -0.04677


In [7]:
# low_granularity_bots = train_bots_fixed_granularity(depth = [1,2], granularity = 3)
# high_granularity_bots = train_bots_fixed_granularity(depth = [1,2], granularity = 10)

## Competition

In [8]:
def run_competition(bots, rounds, report):
    results = []

    for b1 in bots:
        for b2 in bots:
            g1 = b1.root.max_hand_strength
            g2 = b2.root.max_hand_strength
            
            d1 = b1.root.max_turns
            d2 = b2.root.max_turns
            game_depth = max(d1,d2)
            
            c = CompetitionAbstractPoker(deepcopy(b1), deepcopy(b2), max_turns = game_depth, report = False)
            result = c.compete(rounds = rounds)
            results.append([g1,g2,d1,d2,game_depth,result])
            
            if report:
                print(f"{d1} vs {d2}. Game depth {game_depth} Result is {result}")
                
    return results

In [9]:
low_granularity_competition = run_competition(low_granularity_bots, rounds = _COMPETITION_ROUNDS, report = True)

1 vs 1. Game depth 1 Result is -0.0306
1 vs 2. Game depth 2 Result is -0.03629
1 vs 3. Game depth 3 Result is -0.04433
1 vs 4. Game depth 4 Result is -0.07081
2 vs 1. Game depth 2 Result is -0.02667
2 vs 2. Game depth 2 Result is -0.02328
2 vs 3. Game depth 3 Result is -0.0269
2 vs 4. Game depth 4 Result is -0.04342
3 vs 1. Game depth 3 Result is -0.01171
3 vs 2. Game depth 3 Result is -0.01706
3 vs 3. Game depth 3 Result is -0.0302
3 vs 4. Game depth 4 Result is -0.01951
4 vs 1. Game depth 4 Result is -0.00739
4 vs 2. Game depth 4 Result is -0.02077
4 vs 3. Game depth 4 Result is -0.005
4 vs 4. Game depth 4 Result is -0.02326


In [10]:
high_granularity_competition = run_competition(high_granularity_bots, rounds = _COMPETITION_ROUNDS, report = True)

1 vs 1. Game depth 1 Result is -0.05816
1 vs 2. Game depth 2 Result is -0.08096
1 vs 3. Game depth 3 Result is -0.09037
1 vs 4. Game depth 4 Result is -0.12396
2 vs 1. Game depth 2 Result is -0.02658
2 vs 2. Game depth 2 Result is -0.05303
2 vs 3. Game depth 3 Result is -0.07742
2 vs 4. Game depth 4 Result is -0.08104
3 vs 1. Game depth 3 Result is 0.00646
3 vs 2. Game depth 3 Result is -0.03428
3 vs 3. Game depth 3 Result is -0.03355
3 vs 4. Game depth 4 Result is -0.05277
4 vs 1. Game depth 4 Result is 0.02551
4 vs 2. Game depth 4 Result is 0.00168
4 vs 3. Game depth 4 Result is -0.05341
4 vs 4. Game depth 4 Result is -0.04713


# Export results 

In [11]:
low_granularity_df = pd.DataFrame(low_granularity_competition, columns = ['player1_granularity', 'player2_granularity', 
                                                                          'player1_depth', 'player2_depth',
                                                                          'game_depth', 'result'])
high_granularity_df = pd.DataFrame(high_granularity_competition, columns = ['player1_granularity', 'player2_granularity', 
                                                          'player1_depth', 'player2_depth',
                                                          'game_depth', 'result'])

In [12]:
results_df = pd.concat([low_granularity_df, high_granularity_df], ignore_index = True)

In [13]:
results_df.head(3)

,player1_granularity,player2_granularity,player1_depth,player2_depth,game_depth,result
0,3,3,1,1,1,-0.03060
1,3,3,1,2,2,-0.03629
2,3,3,1,3,3,-0.04433


In [14]:
results_df.to_csv("output/bot_competition_by_depth.csv", index = False)